In [27]:
import io
import re
from PyPDF2 import PdfReader
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PdfReader(file)
        full_text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            text = text.replace('\n', ' ')
            text = re.sub(r'\s+', ' ', text)
            full_text += text + ' '
        full_text = full_text.strip()
    return full_text

# Extract text from your PDF file
pdf_text = extract_text_from_pdf('processed.articles.pdf')


In [28]:
# Load the pre-trained BERT model and tokenizer
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
import gc
# Split the text into chunks and tokenize
chunk_size = 1024  # Adjust this value based on your system's memory capacity
chunks = [pdf_text[i:i+chunk_size] for i in range(0, len(pdf_text), chunk_size)]
tokenized_chunks = []


In [30]:
for chunk in chunks:
    tokenized_chunk = tokenizer(chunk, return_tensors='pt', padding=True, truncation=True)
    tokenized_chunks.append(tokenized_chunk)
    del tokenized_chunk  # Free up memory
    gc.collect()  # Collect garbage to free up more memory

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./finetuned_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True
)

# Create the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=data
)




ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('./finetuned_model')

NameError: name 'trainer' is not defined